# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 28 2022 9:51AM,247461,15,VT80178453,"Virtus Pharmaceuticals, LLC",Released
1,Sep 28 2022 9:51AM,247461,15,VT80178454,"Virtus Pharmaceuticals, LLC",Released
2,Sep 28 2022 9:51AM,247461,15,VT80178455,"Virtus Pharmaceuticals, LLC",Released
3,Sep 28 2022 9:51AM,247461,15,VT80178456,"Virtus Pharmaceuticals, LLC",Released
4,Sep 28 2022 9:51AM,247461,15,VT80178457,"Virtus Pharmaceuticals, LLC",Released
5,Sep 28 2022 9:51AM,247461,15,VT80178458,"Virtus Pharmaceuticals, LLC",Released
6,Sep 28 2022 9:51AM,247461,15,VT80178459,"Virtus Pharmaceuticals, LLC",Released
7,Sep 28 2022 9:51AM,247461,15,VT80178460,"Virtus Pharmaceuticals, LLC",Released
8,Sep 28 2022 9:51AM,247461,15,VT80178461,"Virtus Pharmaceuticals, LLC",Released
9,Sep 28 2022 9:51AM,247461,15,VT80178462,"Virtus Pharmaceuticals, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
26,247454,Released,9
27,247457,Released,1
28,247458,Released,2
29,247460,Released,21
30,247461,Released,16


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
247454,NaN,NaN,9.0
247457,NaN,NaN,1.0
247458,NaN,NaN,2.0
247460,NaN,NaN,21.0
247461,NaN,NaN,16.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247202,0.0,1.0,0.0
247205,0.0,1.0,0.0
247385,16.0,2.0,4.0
247386,0.0,0.0,1.0
247403,9.0,6.0,14.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247202,0,1,0
247205,0,1,0
247385,16,2,4
247386,0,0,1
247403,9,6,14


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247202,0,1,0
1,247205,0,1,0
2,247385,16,2,4
3,247386,0,0,1
4,247403,9,6,14


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247202,,1,
1,247205,,1,
2,247385,16,2,4
3,247386,,,1
4,247403,9,6,14


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 28 2022 9:51AM,247461,15,"Virtus Pharmaceuticals, LLC"
16,Sep 28 2022 9:43AM,247460,15,"Brookfield Pharmaceuticals, LLC"
37,Sep 28 2022 9:35AM,247458,10,"Citieffe, Inc."
39,Sep 28 2022 9:34AM,247457,20,"ACI Healthcare USA, Inc."
40,Sep 28 2022 9:30AM,247435,15,"Alliance Pharma, Inc."
71,Sep 28 2022 9:28AM,247454,10,ISDIN Corporation
80,Sep 28 2022 9:28AM,247450,10,ISDIN Corporation
123,Sep 28 2022 9:28AM,247448,10,ISDIN Corporation
170,Sep 28 2022 8:03AM,247439,19,ACG North America LLC
171,Sep 28 2022 7:41AM,247436,19,"AdvaGen Pharma, Ltd"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 28 2022 9:51AM,247461,15,"Virtus Pharmaceuticals, LLC",,,16
1,Sep 28 2022 9:43AM,247460,15,"Brookfield Pharmaceuticals, LLC",,,21
2,Sep 28 2022 9:35AM,247458,10,"Citieffe, Inc.",,,2
3,Sep 28 2022 9:34AM,247457,20,"ACI Healthcare USA, Inc.",,,1
4,Sep 28 2022 9:30AM,247435,15,"Alliance Pharma, Inc.",,,31
5,Sep 28 2022 9:28AM,247454,10,ISDIN Corporation,,,9
6,Sep 28 2022 9:28AM,247450,10,ISDIN Corporation,,,43
7,Sep 28 2022 9:28AM,247448,10,ISDIN Corporation,,,47
8,Sep 28 2022 8:03AM,247439,19,ACG North America LLC,,,1
9,Sep 28 2022 7:41AM,247436,19,"AdvaGen Pharma, Ltd",,,5


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 28 2022 9:51AM,247461,15,"Virtus Pharmaceuticals, LLC",16,,
1,Sep 28 2022 9:43AM,247460,15,"Brookfield Pharmaceuticals, LLC",21,,
2,Sep 28 2022 9:35AM,247458,10,"Citieffe, Inc.",2,,
3,Sep 28 2022 9:34AM,247457,20,"ACI Healthcare USA, Inc.",1,,
4,Sep 28 2022 9:30AM,247435,15,"Alliance Pharma, Inc.",31,,
5,Sep 28 2022 9:28AM,247454,10,ISDIN Corporation,9,,
6,Sep 28 2022 9:28AM,247450,10,ISDIN Corporation,43,,
7,Sep 28 2022 9:28AM,247448,10,ISDIN Corporation,47,,
8,Sep 28 2022 8:03AM,247439,19,ACG North America LLC,1,,
9,Sep 28 2022 7:41AM,247436,19,"AdvaGen Pharma, Ltd",5,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 28 2022 9:51AM,247461,15,"Virtus Pharmaceuticals, LLC",16,,
1,Sep 28 2022 9:43AM,247460,15,"Brookfield Pharmaceuticals, LLC",21,,
2,Sep 28 2022 9:35AM,247458,10,"Citieffe, Inc.",2,,
3,Sep 28 2022 9:34AM,247457,20,"ACI Healthcare USA, Inc.",1,,
4,Sep 28 2022 9:30AM,247435,15,"Alliance Pharma, Inc.",31,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 28 2022 9:51AM,247461,15,"Virtus Pharmaceuticals, LLC",16.0,NaN,NaN
1,Sep 28 2022 9:43AM,247460,15,"Brookfield Pharmaceuticals, LLC",21.0,NaN,NaN
2,Sep 28 2022 9:35AM,247458,10,"Citieffe, Inc.",2.0,NaN,NaN
3,Sep 28 2022 9:34AM,247457,20,"ACI Healthcare USA, Inc.",1.0,NaN,NaN
4,Sep 28 2022 9:30AM,247435,15,"Alliance Pharma, Inc.",31.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 28 2022 9:51AM,247461,15,"Virtus Pharmaceuticals, LLC",16.0,0.0,0.0
1,Sep 28 2022 9:43AM,247460,15,"Brookfield Pharmaceuticals, LLC",21.0,0.0,0.0
2,Sep 28 2022 9:35AM,247458,10,"Citieffe, Inc.",2.0,0.0,0.0
3,Sep 28 2022 9:34AM,247457,20,"ACI Healthcare USA, Inc.",1.0,0.0,0.0
4,Sep 28 2022 9:30AM,247435,15,"Alliance Pharma, Inc.",31.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1979466,109.0,14.0,4.0
15,742356,68.0,0.0,0.0
16,247430,0.0,1.0,0.0
19,1237140,18.0,13.0,14.0
20,989631,20.0,8.0,25.0
21,494407,0.0,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1979466,109.0,14.0,4.0
1,15,742356,68.0,0.0,0.0
2,16,247430,0.0,1.0,0.0
3,19,1237140,18.0,13.0,14.0
4,20,989631,20.0,8.0,25.0
5,21,494407,0.0,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,109.0,14.0,4.0
1,15,68.0,0.0,0.0
2,16,0.0,1.0,0.0
3,19,18.0,13.0,14.0
4,20,20.0,8.0,25.0
5,21,0.0,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,109.0
1,15,Released,68.0
2,16,Released,0.0
3,19,Released,18.0
4,20,Released,20.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20,21
Status,,,,,,
Completed,4.0,0.0,0.0,14.0,25.0,0.0
Executing,14.0,0.0,1.0,13.0,8.0,2.0
Released,109.0,68.0,0.0,18.0,20.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20,21
0,Completed,4.0,0.0,0.0,14.0,25.0,0.0
1,Executing,14.0,0.0,1.0,13.0,8.0,2.0
2,Released,109.0,68.0,0.0,18.0,20.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20,21
0,Completed,4.0,0.0,0.0,14.0,25.0,0.0
1,Executing,14.0,0.0,1.0,13.0,8.0,2.0
2,Released,109.0,68.0,0.0,18.0,20.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()